In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns

import requests
import json

import pybaseball
from pybaseball import statcast
pybaseball.cache.enable()

import os
import glob
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [12]:
import sys
import time

import matplotlib.pyplot as plt

%matplotlib inline
import numpy as np
import pandas as pd
from IPython.display import HTML

sys.path.append("code/.")

import mglearn
from IPython.display import display

# Classifiers and regressors
from sklearn.dummy import DummyClassifier, DummyRegressor

# Preprocessing and pipeline
from sklearn.impute import SimpleImputer

# train test split and cross validation
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    MinMaxScaler,
    OneHotEncoder,
    OrdinalEncoder,
    StandardScaler,
)
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

pd.set_option("display.max_colwidth", 200)

In [3]:
# load year 2024 data to see how the datasets look like 
df_2024_game_info = pd.read_csv('data_2014_to_2024/df_2024_game_info.csv')
df_2024_batting_stats = pd.read_csv('data_2014_to_2024/df_2024_batting_stats.csv')
df_2024_pitching_stats = pd.read_csv('data_2014_to_2024/df_2024_pitching_stats.csv')
print(df_2024_game_info.shape, df_2024_batting_stats.shape, df_2024_pitching_stats.shape)


(2515, 28) (45270, 15) (5030, 12)


In [7]:
df_2024_game_info.iloc[:,:-17].head()

,Unnamed: 0,game_pk,home_result,date,away_team,home_team,away_final_score,home_final_score,away_starting_pitcher,home_starting_pitcher,away_b1
0,0,745444,L,2024-03-20,LAD,SD,5,2,607192,506433,605141
1,1,747889,L,2024-03-20,DET,MIN,12,3,663554,686973,656716
2,2,747888,L,2024-03-20,NYM,MIA,6,3,681035,687287,607043
3,3,747887,W,2024-03-20,PIT,NYY,0,12,594835,596133,665833
4,4,747885,W,2024-03-20,TOR,ATL,9,10,670102,628452,676391


In [8]:
df_2024_batting_stats.head()

,Unnamed: 0,game_sequence,game_pk,team,is_home,batter,batting_order,ab,bb,hbp,single,double,triple,hr,sf
0,0,1,745444,LAD,False,605141,1,4,0,0,2,0,0,0,0
1,1,1,745444,LAD,False,660271,2,5,0,0,2,0,0,0,0
2,2,1,745444,LAD,False,518692,3,2,1,1,0,0,0,0,0
3,3,1,745444,LAD,False,669257,4,4,1,0,1,0,0,0,0
4,4,1,745444,LAD,False,571970,5,3,2,0,1,0,0,0,0


In [9]:
df_2024_pitching_stats.head()

,Unnamed: 0,game_sequence,game_pk,team,starting_pitcher,is_home,n_pitches,ip,er,k,bb,h
0,0,1,745444,LAD,607192,False,77.0,5.00,2.0,3.0,4.0,2.0
1,1,1,745444,SD,506433,True,72.0,3.67,0.0,3.0,3.0,2.0
2,2,2,747889,DET,663554,False,70.0,4.67,0.0,4.0,2.0,2.0
3,3,2,747889,MIN,686973,True,76.0,4.00,8.0,0.0,3.0,9.0
4,4,3,747888,NYM,681035,False,59.0,4.00,1.0,7.0,0.0,2.0


- for batter, do a rolling average of OPS over the last 10 games (impute if the game is in the team's first 10 games) (insert the formula here). 
- for pitcher, do a rolling average of his ERA/WHIP/(BB or K9) over "his" last 10 starts (impute if lacking). 

In [ ]:
def calc_ops(ab,bb,hbp,single,double,triple,hr,sf):
    '''
    calculate on base plus slugging average of a single batter  
    to calculate the rolling average, 
    simply take in the TOTAL VALUE of each param (up to last 10 games)
    '''
    h = single+double+triple+hr
    obp = (h+bb+hbp)/(ab+bb+hbp+sf)
    slg = (single + 2*double + 3*triple + 4*hr)/ab

    return obp+slg

def calc_era(er,ip):
    '''
    calculate earned run average of a single pitcher
    to calculate the rolling average, 
    simply take in the TOTAL VALUE of each param (up to last 10 games)
    '''
    return er/ip

# add more metrics if needed 

In [ ]:
df_2024_pitching_era = df_2024_pitching_stats.copy()

# add feature 
